In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.special import inv_boxcox

In [ ]:
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

def get_test_file_path(image_id):
    return "../input/petfinder-pawpularity-score/test/{}.jpg".format(image_id)

test['file_path'] = test['Id'].apply(get_test_file_path)

display(test.head(2))

In [ ]:
import os

OUTPUT_DIR = './'


models = []

models.append(['../input/pawpularity-weight/output_007_swin_base_patch4_window12_384/', 5, 1, 0.334, False, False, -1, "", 'swin_base_patch4_window12_384'])
models.append(['../input/pawpularity-weight/output_037_swin_large_patch4_window12_384_SAM_seed2020/', 5, 1, 0.333, False, False, -1, "", 'swin_large_patch4_window12_384'])
models.append(['../input/pawpularity-weight/output_000_08_ConvNeXt/', 5, 1, 0.333, True, False, -1, "", 'convnext_xlarge'])

NUM_MODELS = len(models)


all_fold_preds = []
MODEL_DIR = []
NFOLDS = []
MODEL_TYPE = []
WEIGHT = []
IS_SMALL = []
IS_BOXCOX = []
SVR_W = []
SVR_DIR = []
MODEL_NAME = []

for i in range(NUM_MODELS):
    all_fold_preds.append([])
    MODEL_DIR.append(models[i][0])
    NFOLDS.append(models[i][1])
    MODEL_TYPE.append(models[i][2])
    WEIGHT.append(models[i][3])
    IS_SMALL.append(models[i][4])
    IS_BOXCOX.append(models[i][5])
    SVR_W.append(models[i][6])
    SVR_DIR.append(models[i][7])
    MODEL_NAME.append(models[i][8])
    
print(WEIGHT)

In [ ]:
# ====================================================
# CFG
# ====================================================
class CFG:
    num_workers=4
    size=384
    size_small=224
    batch_size=48
    model_name=MODEL_NAME
    seed=42
    target_size=1
    target_col='Pawpularity'
    n_fold=NFOLDS
    model_type=MODEL_TYPE
    weight=WEIGHT
    lmbda=0.43173648666774017

In [ ]:
# ====================================================
# Library
# ====================================================
import os
import gc
import sys
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
import timm

sys.path.append('../input/convnext/ConvNeXt-main')
import models.convnext

sys.path.append('../input/ttach-kaggle/ttach')
import ttach

from torch.cuda.amp import autocast, GradScaler

import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

import cuml, pickle
from cuml.svm import SVR
print('RAPIDS version',cuml.__version__,'\n')

In [ ]:
# ====================================================
# Utils
# ====================================================
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def get_score(y_true, y_pred):
    score = mean_squared_error(y_true * 100.0, sigmoid(y_pred) * 100.0, squared=False) # RMSE
    return score


def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

# LOGGER = init_logger()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

In [ ]:
# ====================================================
# Dataset
# ====================================================
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.file_names[idx]
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image=image)['image']
        return image

In [ ]:
# ====================================================
# Transforms
# ====================================================
def get_transforms(*, data):
    
    if data == 'train':
        return A.Compose([
            A.RandomResizedCrop(CFG.size, CFG.size, scale=(0.85, 1.0)),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

    elif data == 'valid':
        return A.Compose([
            #A.Resize(CFG.size, CFG.size),
            A.Resize(CFG.size + 70, CFG.size + 70),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])   
    elif data == 'valid_small':
        return A.Compose([
            #A.Resize(CFG.size_small, CFG.size_small),
            A.Resize(CFG.size_small + 45, CFG.size_small + 45),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

In [ ]:
# ====================================================
# MODEL
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, idx=0, pretrained=False):
        super().__init__()
        self.cfg = cfg
        self.model = timm.create_model(self.cfg.model_name[idx], pretrained=pretrained)
        if 1 == self.cfg.model_type[idx] or 0 == self.cfg.model_type[idx]:
            self.model.head = nn.Linear(self.model.head.in_features, 128)
            self.dropout = nn.Dropout(0.1)
            self.dense1 = nn.Linear(128, 64)
            self.dense2 = nn.Linear(64, 1)
        elif 2 == self.cfg.model_type[idx]:
            self.model.head = nn.Sequential(
                nn.Dropout(0.5),
                nn.Linear(self.model.head.in_features, 1)
            )

    def feature(self, image):
        feature = self.model(image)
        return feature
        
    def forward(self, image):
        if 1 == self.cfg.model_type[idx]:
            x = self.feature(image)
            x = self.dropout(x)
            x = self.dense1(x)
            x = self.dense2(x)
        elif 2 == self.cfg.model_type[idx]:
            x = self.model(image)
        elif 0 == self.cfg.model_type[idx]:
            feat = self.model(image)
            x = self.dense1(feat)
            x = self.dense2(x)
            x = torch.cat([x, feat], dim=1)
        return x

In [ ]:
# ====================================================
# Helper functions
# ====================================================
def get_features(test_loader, model, device):
    model.eval()
    features = []
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    for step, (images) in tk0:
        images = images.to(device)
        batch_size = images.size(0)
        with torch.no_grad():
            feature = model.feature(images)
        features.append(feature.to('cpu').numpy())
    features = np.concatenate(features)
    return features

def predict(test_loader, model, device):
    model.eval()
    preds = []
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    for step, (images) in tk0:
        images = images.to(device)
        batch_size = images.size(0)
        with torch.no_grad():
            pred = model(images)
        preds.append(pred.to('cpu').numpy())
    preds = np.concatenate(preds)
    return preds

def predict_with_feature(test_loader, model, device):
    model.eval()
    preds = []
    features = []
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    for step, (images) in tk0:
        images = images.to(device)
        batch_size = images.size(0)
        with torch.no_grad():
            pred = model(images)
        pred_cpu = pred.to('cpu').numpy()
        preds.append(pred_cpu[:,0])
        features.append(pred_cpu[:,1:])
    preds = np.concatenate(preds)
    features = np.concatenate(features)
    return preds, features   

In [ ]:
test_dataset = TestDataset(test, transform=get_transforms(data='valid'))
test_loader = DataLoader(test_dataset, 
                         batch_size=CFG.batch_size * 2, 
                         shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

test_dataset_small = TestDataset(test, transform=get_transforms(data='valid_small'))
test_loader_small = DataLoader(test_dataset_small, 
                         batch_size=CFG.batch_size * 4, 
                         shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

for idx in range(len(CFG.model_name)):
    print(CFG.model_name[idx])
    for fold in range(CFG.n_fold[idx]):
        if SVR_W[idx] > 0:
            # LOAD RAPIDS SVR 
            name = f"SVR_fold_{fold}.pkl" 
            print('Loading SVR...',SVR_DIR[idx]+name)
            clf = pickle.load(open(SVR_DIR[idx]+name, "rb"))
            
            model = CustomModel(CFG, idx=idx, pretrained=False)
            print(MODEL_DIR[idx]+f'{CFG.model_name[idx]}_fold{fold}_best.pth')
            state = torch.load(MODEL_DIR[idx]+f'{CFG.model_name[idx]}_fold{fold}_best.pth', 
                               map_location=torch.device('cpu'))['model']
            model.load_state_dict(state)
            model.to(device)
            if IS_SMALL[idx] is True:
                model = ttach.ClassificationTTAWrapper(model, ttach.aliases.five_crop_transform(CFG.size_small, CFG.size_small), merge_mode='mean')
                #model = ttach.ClassificationTTAWrapper(model, ttach.aliases.ten_crop_transform(CFG.size_small, CFG.size_small), merge_mode='mean')
                preds, feats = predict_with_feature(test_loader_small, model, device)
                preds = sigmoid(preds) * 100.0
                
                preds1 = preds.copy()
                preds2 = clf.predict(feats)
                preds = (1 - SVR_W[idx]) * preds1 + SVR_W[idx] * preds2
                all_fold_preds[idx].append(preds)
            else:
                pass
            
        else:
            model = CustomModel(CFG, idx=idx, pretrained=False)
            print(MODEL_DIR[idx]+f'{CFG.model_name[idx]}_fold{fold}_best.pth')
            state = torch.load(MODEL_DIR[idx]+f'{CFG.model_name[idx]}_fold{fold}_best.pth', 
                               map_location=torch.device('cpu'))['model']
            model.load_state_dict(state)
            model.to(device)
            if IS_SMALL[idx] is True:
                #model = ttach.ClassificationTTAWrapper(model, ttach.aliases.five_crop_transform(CFG.size_small, CFG.size_small), merge_mode='mean')
                model = ttach.ClassificationTTAWrapper(model, ttach.aliases.ten_crop_transform(CFG.size_small, CFG.size_small), merge_mode='mean')
                preds = predict(test_loader_small, model, device)
            else:
                #model = ttach.ClassificationTTAWrapper(model, ttach.aliases.flip_transform(), merge_mode='mean')
                #model = ttach.ClassificationTTAWrapper(model, ttach.aliases.five_crop_transform(CFG.size, CFG.size), merge_mode='mean')
                model = ttach.ClassificationTTAWrapper(model, ttach.aliases.ten_crop_transform(CFG.size, CFG.size), merge_mode='mean')
                preds = predict(test_loader, model, device)
            if IS_BOXCOX[idx]:
                all_fold_preds[idx].append(inv_boxcox(preds, CFG.lmbda))
            else:
                all_fold_preds[idx].append(sigmoid(preds) * 100.0)
            del state; gc.collect()
            torch.cuda.empty_cache()

In [ ]:
preds = None
for idx in range(len(CFG.model_name)):
    predictions = np.reshape(np.mean(np.array(all_fold_preds[idx]), 0), (-1, 1))
    if idx == 0:
        preds = predictions
    else:
        preds = np.hstack([preds, predictions])

preds = preds * np.array([CFG.weight])
predictions = np.sum(preds, 1)

test['Pawpularity'] = predictions
test[['Id', 'Pawpularity']].to_csv('submission.csv', index=False)
display(test[['Id', 'Pawpularity']].head())